In [ ]:
# FONTE: Dataset covid19 Brasil.IO
%matplotlib inline
import yaml
import numpy as np
import pandas as pd
import ipywidgets as widgets

from datetime import datetime, date, timedelta
from IPython.display import display
import matplotlib.pyplot as plt

## get rid of annoying warnings
import warnings
warnings.simplefilter('ignore')

URL_BASE =  'https://brasil.io/api/dataset/covid19/caso_full/data/?format=json&is_repeated=False&page_size=10000&date={}'
DATA_DICT={'Deaths':'last_available_deaths', 'Confirmed':'last_available_confirmed', 'New Confirmed':'new_confirmed',
           'New Deaths':'new_deaths', 'Death Rate':'last_available_death_rate', 'Confirmed/100k':'last_available_confirmed_per_100k_inhabitants'}

pd.set_option('display.float_format', lambda x: '%.4f' % x)
def coronavirus_panel():
    output =widgets.Output()
    start_date_widget = widgets.DatePicker(description='Start Date', value=date(2020,2,25))
    end_date_widget = widgets.DatePicker(description='End Date', value=date.today())
    run_button  = widgets.Button(description='Run')
    progress_bar = widgets.FloatProgress(value=0, min=0, max=100, step=0.1, description='Loading', bar_style='info', orientation='horizontal')
    info_widget = widgets.ToggleButtons(options=['Deaths', 'Confirmed', 'New Confirmed', 'New Deaths', 'Death Rate', 'Confirmed/100k'], description='Information')
    death_limiter_widget = widgets.IntSlider(value=0, description='From death #', step=1, min=0, max=100)
    moving_average_widget = widgets.IntSlider(value=1, description='MA-Days', step=1, min=1, max=15)
    states_widget = widgets.HBox([])
    panel = widgets.VBox([widgets.HBox([start_date_widget, end_date_widget, run_button, progress_bar]),widgets.HBox([info_widget, death_limiter_widget, moving_average_widget]), states_widget])
    
    def update_df():
        date_range = pd.date_range(start=start_date_widget.value, end=end_date_widget.value, freq='D')
        l = []
        step = 100/len(date_range)
        for dt in date_range:
            progress_bar.value = progress_bar.value + step
            l.append(pd.json_normalize(pd.read_json(URL_BASE.format(dt.date().isoformat())).results))
        progress_bar.value = 0
        df = pd.concat(l).reset_index()
        df.drop(columns=['index'], inplace=True)
        df = df[pd.isna(df.city)]
        states_widget.children = [widgets.ToggleButton(value=True, description=state) if state in ('RJ', 'SP', 'SC', 'RS', 'PA', 'AM', 'MG', 'PR') else widgets.ToggleButton(value=False, description=state) for state in df.state.unique()]
        
        def filter_dataframe(df, info, states):
            output.clear_output()
            plt.rcParams['figure.figsize'] = [27, 18]
            asw = df.pivot(index='date', columns='state', values=DATA_DICT[info]).ffill().fillna(0)
            if info in ('New Confirmed', 'Death Rate', 'Confirmed/100k'):
                deaths = df.pivot(index='date', columns='state', values=DATA_DICT['Deaths']).ffill().fillna(0)
                aux = asw[deaths>death_limiter_widget.value].rolling(moving_average_widget.value).mean().ffill().reset_index()
                index=aux.apply(lambda y:y.first_valid_index()).fillna(0)
                asw = pd.concat([aux[col][aux[col].index>=index[col]].dropna().reset_index().drop(columns=['index']) for col in aux.columns], axis=1).drop(columns='date').dropna(axis=1, how='all').dropna(axis=0, how='all')
            if states:
                asw = asw[states]
            if info == 'Death Rate':
                asw = 100*asw
            asw.plot(title = f"{info} - {start_date_widget.value.isoformat()} to {end_date_widget.value.isoformat()}")
            with output:
                x = plt.plot(data=df)
                plt.ylabel(info, fontsize=30)
                if info in ('New Confirmed', 'Death Rate', 'Confirmed/100k'):
                    plt.xlabel(f'Days from #{moving_average_widget.value} death', fontsize=30)
                else:
                    plt.xlabel(f'Dates', fontsize=30)
                plt.title(f"{info} - {start_date_widget.value.isoformat()} to {end_date_widget.value.isoformat()}", fontsize=30)
                plt.set_cmap('winter')
                plt.legend(fontsize=15, bbox_to_anchor=(1.07,1))
                plt.tick_params(axis='both', labelsize=15)
                plt.show()
        
        filter_dataframe(df, info_widget.value, states=[w.description for w in states_widget.children if w.value]) 
        
        def filter_eventhandler(obj):
            filter_dataframe(df, info_widget.value, states=[w.description for w in states_widget.children if w.value])
        
        for w in states_widget.children:
            w.observe(filter_eventhandler, 'value')
        
        info_widget.observe(filter_eventhandler, 'value')
        death_limiter_widget.observe(filter_eventhandler, 'value')
        moving_average_widget.observe(filter_eventhandler, 'value')
    
    def change_eventhandler(obj):
        update_df()

    run_button.on_click(change_eventhandler)
    return widgets.AppLayout(header=panel, left_sidebar=None,
                             center=output, right_sidebar=None, footer=None, pane_heights=['100px', '1200px', 8])
display(coronavirus_panel())